<a href="https://colab.research.google.com/github/SnoopiACK/DientesMask/blob/master/Diferentes_clases_Detectron2_LABELME_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MaskRCNN training for detecting teeth instances and classes



In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!gcc --version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [0]:
# Some basic setup:
# import some common libraries
import numpy as np
import cv2
import random
import os
import glob
import json
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode

import torch, torchvision

# Train on a custom dataset

In [3]:
#cloning repo
repo_url = 'https://github.com/SnoopiACK/DientesMask'
%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
Cloning into 'DientesMask'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 701 (delta 44), reused 79 (delta 14), pack-reused 557
Receiving objects: 100% (701/701), 299.22 MiB | 14.48 MiB/s, done.
Resolving deltas: 100% (232/232), done.
Checking out files: 100% (444/444), done.
/content/DientesMask
Already up to date.


In [0]:
from os import listdir
from os.path import isfile, join

def get_dientes_dicts(jsons_path, images_path):
    # json_file = os.path.join(img_dir, "via_region_data.json")
    jsons = [f for f in listdir(jsons_path) if isfile(join(jsons_path, f)) and (".json" in f)]
    
    clasesDientes= {"molares"   : 0,
                    "premolares": 1,
                    "caninos"   : 2,
                    "incisivos" : 3
                   }

    dataset_dicts = []
    for idx, json_file in enumerate(jsons):
        
        
        with open(jsons_path+'/'+json_file) as f:
            diccionario = json.load(f)
        
        if not "shapes" in diccionario.keys():
            print(json_file," NO ES UN JSON VALIDO")
            continue
        
        record = {}
        filename = os.path.join(images_path, diccionario['imagePath'])
        print(diccionario['imagePath'])
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = diccionario['imageHeight']
        record["width"] = diccionario['imageWidth']
      
        objs = []
        
        
        for shape in diccionario['shapes']:
            clase_id=shape['label']
            clase_id=clasesDientes[clase_id]

            xy=shape['points']
            px = [x[0] for x in xy]
            py = [y[1] for y in xy]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                    "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                    "bbox_mode": BoxMode.XYXY_ABS,  
                    "segmentation": [poly],
                    "category_id": clase_id,
                    "iscrowd": 0
                }
            objs.append(obj)
        
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [0]:
for d in ["train", "test"]:
    try:
        DatasetCatalog.register("dientes_" + d, lambda d=d: get_dientes_dicts("data/jsons_comparacion_de_redes/jsons_segmentacion/" + d,"data/images/" ))
        MetadataCatalog.get("dientes_" + d).set(thing_classes=["M","P","C","I"])
    except:
        1==1
dientes_metadata = MetadataCatalog.get("dientes_train")

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [0]:
dataset_dicts = get_dientes_dicts("data/jsons_comparacion_de_redes/jsons_segmentacion/train", "data/images")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=dientes_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

## Train!

In [0]:
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

class CustomTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("custom_eval", exist_ok=True)
        output_folder = "custom_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [8]:
%rm -rf output
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dientes_train",)
cfg.DATASETS.TEST = ("dientes_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512#128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  #  Clases molares, premolares, caninos y incisivos
cfg.TEST.EVAL_PERIOD = 10

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[05/23 16:19:09 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:15, 11.4MB/s]                           


Se truncaron las últimas líneas 5000 del resultado de transmisión.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.376
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.730
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.294
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.049
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.393
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.528
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.584
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.750
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.516
[05/23 16:38:22 d2.evaluation.coco_evaluation]: Evaluation results for segm: 
|   AP   |  AP50  |  AP75  |  APs   |  APm   |  APl   |
|:------:|:------:|:------:|:------:|:------:|:------:|
| 29.892 | 53.319 | 26.

In [0]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [0]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#print(cfg.MODEL.WEIGHTS)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("dientes_test", )
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [0]:
  from detectron2.utils.visualizer import ColorMode
  dataset_dicts = get_dientes_dicts("data/jsons_comparacion_de_redes/jsons_segmentacion/test","data/images/" )
  for d in random.sample(dataset_dicts, 6):    
      im = cv2.imread(d["file_name"])
      outputs = predictor(im)
      v = Visualizer(im[:, :, ::-1],
                    metadata=dientes_metadata, 
                    scale=0.5, 
                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
      )
      v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      cv2_imshow(v.get_image()[:, :, ::-1])

We can also evaluate its performance using AP metric implemented in COCO API.

In [0]:
from detectron2.utils.visualizer import ColorMode
print("/content/DientesMask/data/segmentacion/val/nino.jpg")
im = cv2.imread("/content/DientesMask/data/segmentacion/val/nino.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                    metadata=dientes_metadata, 
                    scale=1, 
                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
      )
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [12]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("dientes_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "dientes_test")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

CENZORI_YAMILA_09082018_1327471.jpg
GRIPPO_DANTE_08022017_150207.png
WIRSKE_FLORENCIA_18122018_125834.jpg
AGUIRRE_AGUSTIN_19102018_133949.jpg
STREITEMBERGER_LUCIA_06112017_1135839.jpg
NISI_NATALIA_____1___PM_15112017_171714.jpg
[05/23 17:08:41 d2.data.common]: Serializing 6 elements to byte tensors and concatenating them all ...
[05/23 17:08:41 d2.data.common]: Serialized dataset takes 0.07 MiB
[05/23 17:08:41 d2.evaluation.evaluator]: Start inference on 6 images
[05/23 17:08:49 d2.evaluation.evaluator]: Total inference time: 0:00:01.474989 (1.474989 s / img per device, on 1 devices)
[05/23 17:08:49 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.529563 s / img per device, on 1 devices)
[05/23 17:08:49 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/23 17:08:49 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/23 17:08:49 d2.evaluation.coco_evaluation]: Evaluating predictions ...
Loading and prep

OrderedDict([('bbox',
              {'AP': 70.67463570765563,
               'AP-C': 65.04093339138653,
               'AP-I': 70.36693673932406,
               'AP-M': 74.06699253219432,
               'AP-P': 73.22368016771765,
               'AP50': 96.3243146005245,
               'AP75': 90.93827258445519,
               'APl': 70.01608118665142,
               'APm': 66.03818953323903,
               'APs': 81.12682696841112}),
             ('segm',
              {'AP': 70.08665859552242,
               'AP-C': 69.93170298866268,
               'AP-I': 71.18522044536549,
               'AP-M': 69.7399722142492,
               'AP-P': 69.48973873381229,
               'AP50': 96.3243146005245,
               'AP75': 88.28981898980224,
               'APl': 68.69587540564078,
               'APm': 68.02805280528052,
               'APs': 86.0437293729373})])

In [13]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)
#%mkdir "/gdrive/My Drive/models/faster_rcnn" #o el directorio que se quiera
%cp -r "output/." "/gdrive/My Drive/models/mask_rcnn"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
